# Tutorial: Análisis de Calidad del Aire en Coyhaique

## 1. Introducción

En este tutorial, analizaremos la calidad del aire en Coyhaique utilizando datos del Sistema de Información Nacional de Calidad del Aire (SINCA) de Chile. Nos centraremos en la estación de monitoreo de Coyhaique I.

## 2. Obtención de Datos

Los datos se pueden descargar desde la página del SINCA para la estación Coyhaique I:
[https://sinca.mma.gob.cl/index.php/estacion/index/key/B04](https://sinca.mma.gob.cl/index.php/estacion/index/key/B04)

Pasos para descargar los datos:

1. Ve a la sección "Descarga de Datos" en la página.
2. Selecciona el rango de fechas deseado (por ejemplo, el último año completo).
3. Descarga los archivos CSV para cada contaminante (MP2.5, MP10, O3, CO).
4. Guarda todos los archivos en una carpeta llamada "datos_coyhaique" en tu directorio de trabajo.

Una ves descargados los datos los archivos deberas nombrarlos como 

   - 'MP25.csv',
   - 'MP10.csv',
   - 'O3.csv',
   - 'CO.csv'

y colocarlos en el directorio donde trabajaras. Puedes tambien cargarlos en un archivo pero deberas indicarle al programa donde se encuentran los archivos. 


## 3. Preparación del Entorno

Primero, asegúrate de tener instaladas las bibliotecas necesarias. Puedes instalarlas con pip:

```bash
pip install pandas numpy matplotlib seaborn scikit-learn
```

## 4. Carga y Preparación de Datos



### 4.1 Descargar y cargar los datos:

Estas lineas de código son capaces de leer los archivos descargados y unirlos en un unico archivo que denominaresmos 'contaminante_coyhaique.csv'


``` Python 
import pandas as pd
import os
from datetime import datetime


def leer_archivo_contaminante(ruta_archivo, nombre_contaminante):
    # Leer el archivo CSV
    df = pd.read_csv(ruta_archivo, sep=';', 
                     names=['FECHA', 'HORA', 'Registros_validados', 'Registros_preliminares', 'Registros_no_validados'],
                     parse_dates=[['FECHA', 'HORA']])
    
    # Convertir la columna combinada FECHA_HORA al formato correcto
    df['FECHA_HORA'] = pd.to_datetime(df['FECHA_HORA'], format='%y%m%d %H%M')
    
    # Establecer FECHA_HORA como índice
    df.set_index('FECHA_HORA', inplace=True)
    
    # Renombrar la columna de registros validados con el nombre del contaminante
    df.rename(columns={'Registros_validados': nombre_contaminante}, inplace=True)
    
    # Seleccionar solo la columna del contaminante
    return df[[nombre_contaminante]]

# Directorio donde se encuentran los archivos de datos
directorio_datos = './' # los datos dedeberia estar en el presente directorio

# Lista de contaminantes y sus archivos correspondientes
contaminantes = {
    'MP25': 'MP25.csv',
    'MP10': 'MP10.csv',
    'O3': 'O3.csv',
    'CO': 'CO.csv'
}

# Leer y unir todos los archivos
dfs = []
for contaminante, archivo in contaminantes.items():
    ruta_completa = os.path.join(directorio_datos, archivo)
    if os.path.exists(ruta_completa):
        df = leer_archivo_contaminante(ruta_completa, contaminante)
        dfs.append(df)
    else:
        print(f"Archivo no encontrado: {archivo}")

# Unir todos los DataFrames
df_unido = pd.concat(dfs, axis=1)

# Rellenar valores NaN con 0 o puedes elegir otro método
df_unido.fillna(0, inplace=True)

# Mostrar las primeras filas del DataFrame unido
print(df_unido.head())

# Guardar el DataFrame unido en un nuevo archivo CSV
df_unido.to_csv('datos_calidad_aire_unidos.csv')

print("Datos unidos y guardados en 'datos_calidad_aire_unidos.csv'")

# Información adicional sobre el DataFrame
print("\nInformación del DataFrame unido:")
print(df_unido.info())

print("\nEstadísticas descriptivas:")
print(df_unido.describe())

df_unido.to_csv(f'./contaminante_coyhaique.csv', sep=';', decimal=',')
```

### 4.2 preparar los datos:


```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Función para cargar y preparar datos
def cargar_datos(contaminante):
    df = pd.read_csv(f'./contaminante_coyhaique.csv', sep=';', decimal=',')
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    df = df.rename(columns={'Valor': contaminante})
    return df[['Fecha', contaminante]]

# Cargar datos para cada contaminante
contaminantes = ['MP2.5', 'MP10', 'O3', 'CO']
datos = {}
for cont in contaminantes:
    datos[cont] = cargar_datos(cont)

# Combinar todos los datos en un solo DataFrame
df_combined = datos['MP2.5']
for cont in contaminantes[1:]:
    df_combined = pd.merge(df_combined, datos[cont], on='Fecha', how='outer')

# Mostrar las primeras filas y la información del DataFrame
print(df_combined.head())
print(df_combined.info())
```

## 5. Limpieza y Preparación de Datos

Ahora, vamos a limpiar y preparar los datos para el análisis:

```python
# Manejar valores nulos
print("\nValores nulos antes de la limpieza:")
print(df_combined.isnull().sum())

# Eliminar filas con valores nulos
df_clean = df_combined.dropna()

# Filtrar valores extremos (por ejemplo, MP2.5 > 1000)
df_clean = df_clean[df_clean['MP2.5'] <= 1000]

print("\nShape después de la limpieza:", df_clean.shape)

# Agregar columna de estación del año
df_clean['estacion'] = df_clean['Fecha'].dt.month.map({12:0, 1:0, 2:0, 3:1, 4:1, 5:1, 6:2, 7:2, 8:2, 9:3, 10:3, 11:3})
df_clean['estacion'] = df_clean['estacion'].map({0:'Verano', 1:'Otoño', 2:'Invierno', 3:'Primavera'})

print("\nPrimeras filas del dataset limpio:")
print(df_clean.head())
```

## 6. Análisis Exploratorio de Datos

Vamos a realizar algunas visualizaciones para entender mejor nuestros datos:

```python
# Serie temporal de MP2.5
plt.figure(figsize=(12, 6))
plt.plot(df_clean['Fecha'], df_clean['MP2.5'])
plt.title('Niveles de MP2.5 en Coyhaique a lo largo del tiempo')
plt.xlabel('Fecha')
plt.ylabel('MP2.5 (µg/m³)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Boxplot de MP2.5 por estación del año
plt.figure(figsize=(10, 6))
sns.boxplot(x='estacion', y='MP2.5', data=df_clean)
plt.title('Distribución de MP2.5 por Estación del Año')
plt.ylabel('MP2.5 (µg/m³)')
plt.show()

# Matriz de correlación
corr = df_clean[contaminantes].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Matriz de Correlación de Contaminantes')
plt.show()
```

## 7. Modelado Predictivo

Vamos a crear un modelo de regresión lineal simple para predecir los niveles de MP2.5:

```python
# Preparar los datos para el modelo
X = df_clean[['MP10',  'O3', 'CO']]
y = df_clean['MP2.5']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo
modelo = LinearRegression()
modelo.fit(X_train, y_train)

# Hacer predicciones
y_pred = modelo.predict(X_test)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Error cuadrático medio: {mse}")
print(f"R² Score: {r2}")

# Visualizar predicciones vs valores reales
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('MP2.5 Real')
plt.ylabel('MP2.5 Predicho')
plt.title('Predicciones vs Valores Reales de MP2.5')
plt.show()
```

## 8. Análisis Adicionales

Aquí hay algunos análisis adicionales que puedes realizar:

```python
# Variación diaria promedio de MP2.5
df_clean['hora'] = df_clean['Fecha'].dt.hour
hourly_avg = df_clean.groupby('hora')['MP2.5'].mean()

plt.figure(figsize=(10, 6))
plt.plot(hourly_avg.index, hourly_avg.values)
plt.title('Variación Diaria Promedio de MP2.5')
plt.xlabel('Hora del Día')
plt.ylabel('MP2.5 Promedio (µg/m³)')
plt.show()

# Comparación entre días laborables y fines de semana
df_clean['dia_semana'] = df_clean['Fecha'].dt.dayofweek
df_clean['tipo_dia'] = df_clean['dia_semana'].map(lambda x: 'Fin de semana' if x >= 5 else 'Día laborable')

plt.figure(figsize=(10, 6))
sns.boxplot(x='tipo_dia', y='MP2.5', data=df_clean)
plt.title('MP2.5: Días Laborables vs Fines de Semana')
plt.ylabel('MP2.5 (µg/m³)')
plt.show()
```

## 9. Interpretación y Conclusiones

Basándote en los resultados obtenidos, responde las siguientes preguntas:

1. ¿En qué estación del año se observan los niveles más altos de MP2.5 en Coyhaique?
2. ¿Existe una correlación fuerte entre MP2.5 y algún otro contaminante?
3. ¿Qué tan bien predice nuestro modelo de regresión lineal los niveles de MP2.5?
4. ¿Hay diferencias significativas en los niveles de MP2.5 entre días laborables y fines de semana?
5. ¿Qué factores podrían estar contribuyendo a los patrones observados en la calidad del aire de Coyhaique?

## 10. Ejercicios Adicionales

1. Intenta usar otro algoritmo de aprendizaje automático (por ejemplo, Random Forest) para predecir MP2.5 y compara su rendimiento con la regresión lineal.
2. Investiga si hay una relación entre los niveles de contaminación y las condiciones meteorológicas (si tienes acceso a datos meteorológicos).
3. Crea un mapa de calor que muestre los niveles de MP2.5 por hora del día y mes del año.

## Conclusión

Este análisis nos ha permitido explorar en profundidad los patrones de contaminación del aire en Coyhaique, con un enfoque particular en las partículas MP2.5. Hemos identificado tendencias estacionales, correlaciones entre contaminantes y creado un modelo predictivo simple.

Recuerda que la contaminación del aire es un problema complejo influenciado por múltiples factores, incluyendo condiciones meteorológicas, topografía y actividades humanas. Este análisis proporciona una base para entender mejor la situación en Coyhaique, pero se necesitaría un estudio más profundo para obtener conclusiones definitivas y proponer soluciones efectivas.

